In [1]:
# Cell 1: importy i konfiguracja ścieżki do modułów projektu
import json
from pathlib import Path
import sys

import yaml  # type: ignore

# Upewniamy się, że katalog "src" (z modułami projektu) jest na ścieżce importów
src_path = Path("src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from axiomatic_kernel import (
    AxiomKernel,
    VariableSchema,
    AxiomDefinition,
    DecisionLogger,
)
from nl_rule_parser import build_axiom_from_nl, RuleParseError  # noqa: F401
from explanation_engine import DecisionExplainer, ExplanationConfig
from rules_io import load_ruleset_from_file  # noqa: F401
from ruleset_manager import RulesetRegistry, Environment
from rule_analytics import RuleAnalyticsEngine


In [2]:
# Cell 2: konfiguracja schematu, kernela i wczytanie rulesetu FRAUD z governance (FAZA 2.2)

# === 1) SCHEMA DOPASOWANA DO FRAUD_RULES (tylko typy obsługiwane przez kernel) ===
schema = [
    VariableSchema("amount", "int", "Kwota transakcji w jednostkach minimalnych."),
    VariableSchema("tx_count_24h", "int", "Liczba transakcji w ostatnich 24h."),
    VariableSchema("is_pep", "bool", "Czy klient jest PEP."),
    VariableSchema("is_suspicious", "bool", "Czy transakcja jest podejrzana."),
]

# Katalog na logi decyzji
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)

logger = DecisionLogger(logs_dir / "fraud_rules_demo.jsonl")

kernel = AxiomKernel(
    schema=schema,
    decision_variable="is_suspicious",
    logger=logger,
    rule_version="fraud_rules_v1",  # zostanie nadpisane przez RulesetRegistry
)

# === 2) Wczytanie i rejestracja rulesetu FRAUD w środowisku DEV ===
rules_dir = Path("rules")
rules_dir.mkdir(exist_ok=True)

fraud_rules_path = rules_dir / "fraud_rules_v1.yaml"

# Zakładamy, że plik fraud_rules_v1.yaml istnieje w katalogu "rules".
# Jeśli chcesz, możesz tu dodać kod, który go stworzy przy pierwszym uruchomieniu.

registry = RulesetRegistry()

record = registry.register_ruleset(
    ruleset_id="fraud_rules_v1",
    path=fraud_rules_path,
    environment=Environment.DEV,
)

print(
    f"📘 Zarejestrowano ruleset: {record.key.ruleset_id} "
    f"(v{record.version}) w środowisku {record.key.environment.value}"
)
print(f"Plik: {record.file_path}")

# === 3) Nałożenie rulesetu na kernel przez RulesetRegistry ===
summary = registry.apply_ruleset_to_kernel(
    ruleset_id="fraud_rules_v1",
    environment=Environment.DEV,
    kernel=kernel,
    schema=schema,
    decision_field_fallback="is_suspicious",
    strict=True,
    extra_metadata={"domain": "fraud-demo"},
)

print("\n📊 Podsumowanie ładowania reguł:")
print(f"- total_rules:   {summary.total_rules}")
print(f"- enabled_rules: {summary.enabled_rules}")
print(f"- loaded_rules:  {summary.loaded_rules}")
print(f"- skipped_rules: {summary.skipped_rules}")
print(f"- errors:        {summary.errors}")


📘 Zarejestrowano ruleset: fraud_rules_v1 (v1.0.0) w środowisku DEV
Plik: rules/fraud_rules_v1.yaml

📊 Podsumowanie ładowania reguł:
- total_rules:   4
- enabled_rules: 4
- loaded_rules:  4
- skipped_rules: 0
- errors:        {}


In [3]:
# Cell 3: przykładowe case'y FRAUD (FLAGGED / CLEAN) + wyjaśnienia

explainer = DecisionExplainer(ExplanationConfig(language="pl"))

# Przypadek "podejrzany" – przykład, który wg rulesetu powinien być oflagowany
case_flagged = {
    "amount": 15_000,
    "tx_count_24h": 20,
    "is_pep": True,
}

bundle_flagged = kernel.evaluate(case_flagged)

print("=== RAW BUNDLE (FLAGGED) ===")
print(json.dumps(bundle_flagged, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (FLAGGED) ===")
print(explainer.explain(bundle_flagged).to_text(language="pl"))

# Przypadek "czysty" – przykład, który wg rulesetu powinien być CLEAN
case_clean = {
    "amount": 500,
    "tx_count_24h": 1,
    "is_pep": False,
}

bundle_clean = kernel.evaluate(case_clean)

print("\n=== RAW BUNDLE (CLEAN) ===")
print(json.dumps(bundle_clean, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (CLEAN) ===")
print(explainer.explain(bundle_clean).to_text(language="pl"))


=== RAW BUNDLE (FLAGGED) ===
{
  "decision_status": "SAT",
  "decision": "FLAGGED",
  "facts": {
    "amount": 15000,
    "tx_count_24h": 20,
    "is_pep": true
  },
  "model": {
    "amount": 15000,
    "tx_count_24h": 20,
    "is_pep": true,
    "is_suspicious": true
  },
  "satisfied_axioms": [
    {
      "id": "fraud.high_amount",
      "description": "IF amount > 10000 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": true
    },
    {
      "id": "fraud.velocity",
      "description": "IF tx_count_24h > 5 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": true
    },
    {
      "id": "fraud.ultra_high_amount_extreme_velocity",
      "description": "IF amount > 1000000 AND tx_count_24h > 100 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": false
    },
    {
      "id": "fraud.pep_high_risk",
      "description": "If is_pep == true and amount > 2000 then is_suspicious = true",
      "holds": true,
      "ant

In [4]:
# Cell 4: demo UNSAT – sprzeczne reguły na osobnym kernelu (niezależne od FRAUD)

from z3 import Implies  # type: ignore

unsat_schema = [
    VariableSchema("amount", "int", "Kwota transakcji (demo UNSAT)."),
    VariableSchema("risk_score", "int", "Pole demo – nieużywane w regułach."),
    VariableSchema("flag", "bool", "Decyzja testowa (demo UNSAT)."),
]

unsat_kernel = AxiomKernel(
    schema=unsat_schema,
    decision_variable="flag",
    logger=None,
    rule_version="demo_unsat_v1",
)


def rule_flag_true(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == True)


def rule_flag_false(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == False)


unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_true",
        description="If amount > 10000 then flag must be True.",
        build_constraint=rule_flag_true,
    )
)
unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_false",
        description="If amount > 10000 then flag must be False.",
        build_constraint=rule_flag_false,
    )
)

case_conflict = {"amount": 15_000, "risk_score": 5}
bundle_unsat = unsat_kernel.evaluate(case_conflict)

print("=== RAW BUNDLE (UNSAT) ===")
print(json.dumps(bundle_unsat, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (UNSAT) ===")
print(explainer.explain(bundle_unsat).to_text(language="pl"))


=== RAW BUNDLE (UNSAT) ===
{
  "decision_status": "UNSAT",
  "decision": "ERROR",
  "facts": {
    "amount": 15000,
    "risk_score": 5
  },
  "model": {},
  "satisfied_axioms": [],
  "violated_axioms": [],
  "active_axioms": [],
  "inactive_actions": [],
  "conflicting_axioms": [
    "amount_flag_false",
    "amount_flag_true"
  ],
  "rule_version": "demo_unsat_v1",
  "error": "Constraints are unsatisfiable for given case."
}

=== WYJAŚNIENIE (UNSAT) ===
Decyzja niemożliwa: zestaw reguł jest SPRZECZNY dla tego przypadku (UNSAT). Kluczowe dane wejściowe: amount=15000, risk_score=5.

Konflikty reguł:
- Konflikt między regułami: amount_flag_false, amount_flag_true.

Błąd techniczny: Constraints are unsatisfiable for given case.


In [5]:
# Cell 5: przygotowanie rozszerzonego pliku CSV z transakcjami

import csv
from random import Random

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

input_path = data_dir / "transactions_demo.csv"

# Startujemy od ręcznie dobranych 8 case'ów (jak wcześniej)
rows = [
    # transaction_id, customer_id, amount, tx_count_24h, is_pep
    ("T0001", "C0001", 15_000, 1, False),   # wysoka kwota, normalny klient
    ("T0002", "C0002", 500,    1, False),   # mała kwota, mało transakcji (CLEAN)
    ("T0003", "C0003", 500,    10, False),  # velocity (dużo transakcji)
    ("T0004", "C0004", 3_000,  2, True),    # PEP + średnia kwota
    ("T0005", "C0005", 20_000, 8, True),    # PEP + bardzo wysoka kwota + velocity
    ("T0006", "C0006", 8_000,  0, False),   # średnia kwota, brak velocity
    ("T0007", "C0007", 2_500,  6, False),   # tylko velocity
    ("T0008", "C0008", 1_000,  7, True),    # PEP, ale bardzo mała kwota
]

rng = Random(42)
next_index = len(rows) + 1

# Dodajemy ~200 losowych transakcji w różnych segmentach
for i in range(next_index, next_index + 200):
    scenario = rng.random()

    if scenario < 0.25:
        # Scenariusz: wysoka kwota, mało transakcji (high_amount)
        amount = rng.randint(11_000, 50_000)
        tx_count_24h = rng.randint(0, 3)
        is_pep = False
    elif scenario < 0.5:
        # Scenariusz: velocity – dużo transakcji, mała/średnia kwota
        amount = rng.randint(200, 5_000)
        tx_count_24h = rng.randint(6, 15)
        is_pep = False
    elif scenario < 0.7:
        # Scenariusz: PEP high risk – PEP + wyższa kwota
        amount = rng.randint(3_000, 25_000)
        tx_count_24h = rng.randint(0, 5)
        is_pep = True
    elif scenario < 0.9:
        # Scenariusz: raczej czyste – średnia kwota, mało transakcji
        amount = rng.randint(200, 9_000)
        tx_count_24h = rng.randint(0, 3)
        is_pep = False
    else:
        # Scenariusz: miks / szum, ale poniżej progu ultra-high rule
        amount = rng.randint(1_000, 200_000)
        tx_count_24h = rng.randint(0, 20)
        is_pep = rng.random() < 0.3

    rows.append(
        (f"T{i:04d}", f"C{i:04d}", amount, tx_count_24h, is_pep)
    )

with input_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(
        ["transaction_id", "customer_id", "amount", "tx_count_24h", "is_pep"]
    )
    writer.writerows(rows)

print(f"Zapisano rozszerzony plik CSV z transakcjami: {input_path}")
print(f"Liczba transakcji: {len(rows)}")


Zapisano rozszerzony plik CSV z transakcjami: data/transactions_demo.csv
Liczba transakcji: 208


In [6]:
# Cell 5: Rule Analytics – analiza logów FRAUD (FAZA 4, opcjonalnie)
import csv

input_path = data_dir / "transactions_demo.csv"
output_path = data_dir / "transactions_with_explanations.csv"

explainer = DecisionExplainer(ExplanationConfig(language="pl"))

print(f"Czytam dane z: {input_path}")

output_rows = []

with input_path.open("r", newline="", encoding="utf-8") as f_in:
    reader = csv.DictReader(f_in)
    for row in reader:
        # Budujemy case zgodnie ze schema FRAUD:
        #   amount: int
        #   tx_count_24h: int
        #   is_pep: bool
        case = {
            "amount": int(row["amount"]),
            "tx_count_24h": int(row["tx_count_24h"]),
            "is_pep": str(row["is_pep"]).lower() in {"true", "1", "yes", "y", "t"},
        }

        bundle = kernel.evaluate(case)
        logger.log(bundle)  # zapis do logs/fraud_rules_demo.jsonl

        decision = bundle.get("decision")
        status = bundle.get("decision_status")

        # Lista aktywnych reguł (tych, które faktycznie "odpaliły" w tej decyzji)
        active_rules = [ax["id"] for ax in bundle.get("active_axioms", [])]

        # Pełne wyjaśnienie tekstowe po polsku dla tej konkretnej transakcji
        explanation_text = explainer.explain(bundle).to_text(language="pl")

        output_rows.append(
            {
                "transaction_id": row["transaction_id"],
                "customer_id": row["customer_id"],
                "amount": row["amount"],
                "tx_count_24h": row["tx_count_24h"],
                "is_pep": row["is_pep"],
                "decision": decision,
                "decision_status": status,
                "active_rules": ",".join(active_rules),
                "explanation_pl": explanation_text,
            }
        )

# Zapisujemy wynikowy plik CSV z decyzjami i pełnym wyjaśnieniem
with output_path.open("w", newline="", encoding="utf-8") as f_out:
    fieldnames = [
        "transaction_id",
        "customer_id",
        "amount",
        "tx_count_24h",
        "is_pep",
        "decision",
        "decision_status",
        "active_rules",
        "explanation_pl",
    ]
    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_rows)

print(f"\n✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami: {output_path}")

print("\nPodgląd pierwszych 2 wierszy wynikowych:")
for row in output_rows[:2]:
    print("\n---------------------------")
    print(f"TX {row['transaction_id']} | decyzja: {row['decision']} ({row['decision_status']})")
    print("Aktywne reguły:", row["active_rules"] or "–")
    print("Wyjaśnienie:")
    print(row["explanation_pl"])


Czytam dane z: data/transactions_demo.csv

✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami: data/transactions_with_explanations.csv

Podgląd pierwszych 2 wierszy wynikowych:

---------------------------
TX T0001 | decyzja: FLAGGED (SAT)
Aktywne reguły: fraud.high_amount
Wyjaśnienie:
Decyzja: transakcja została OFLAGOWANA (FLAGGED). Kluczowe dane wejściowe: amount=15000, is_pep=False, tx_count_24h=1.

Powody (aktywne reguły):
- Reguła 'fraud.high_amount': IF amount > 10000 THEN is_suspicious = TRUE

Reguły, które nie zadziałały w tym przypadku:
- Reguła 'fraud.velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_24h > 5 THEN is_suspicious = TRUE
- Reguła 'fraud.ultra_high_amount_extreme_velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF amount > 1000000 AND tx_count_24h > 100 THEN is_suspicious = TRUE
- Reguła 'fraud.pep_high_risk' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: If is_pe

In [7]:
analytics_engine = RuleAnalyticsEngine()

analytics_result = analytics_engine.analyze_log_file(
    log_path=logs_dir / "fraud_rules_demo.jsonl",
    ruleset_path=fraud_rules_path,
)

report = analytics_result.as_dict()

print("=== STATYSTYKI DECYZJI ===")
print(json.dumps(report["outcome_stats"], indent=2, ensure_ascii=False))

print("\n=== STATYSTYKI REGUŁ ===")
for rule_id, stats in sorted(report["rule_stats"].items()):
    print(f"\nReguła: {rule_id}")
    print(json.dumps(stats, indent=2, ensure_ascii=False))

coverage = report.get("coverage_report")
if coverage is not None:
    print("\n=== POKRYCIE RULESETU ===")
    print(json.dumps(coverage, indent=2, ensure_ascii=False))


=== STATYSTYKI DECYZJI ===
{
  "total_decisions": 1449,
  "by_decision": {
    "FLAGGED": 1121,
    "CLEAN": 328
  },
  "by_status": {
    "SAT": 1449
  },
  "by_rule_version": {
    "fraud_rules_v1": 4,
    "fraud_rules_v1:1.0.0@DEV": 1445
  },
  "unsat_cases": 0,
  "error_cases": 0
}

=== STATYSTYKI REGUŁ ===

Reguła: fraud.high_amount
{
  "rule_id": "fraud.high_amount",
  "description": "IF amount > 10000 THEN is_suspicious = TRUE",
  "total_occurrences": 1449,
  "satisfied": 1449,
  "violated": 0,
  "active": 655,
  "inactive": 794,
  "in_conflict": 0
}

Reguła: fraud.pep_high_risk
{
  "rule_id": "fraud.pep_high_risk",
  "description": "If is_pep == true and amount > 2000 then is_suspicious = true",
  "total_occurrences": 1435,
  "satisfied": 1435,
  "violated": 0,
  "active": 311,
  "inactive": 1124,
  "in_conflict": 0
}

Reguła: fraud.ultra_high_amount_extreme_velocity
{
  "rule_id": "fraud.ultra_high_amount_extreme_velocity",
  "description": "IF amount > 1000000 AND tx_count_24

In [8]:
# Cell X: Raport AML na podstawie pliku CSV z wyjaśnieniami
# oraz zapis tego raportu do pliku TXT

import csv

# Upewniamy się, że mamy katalog z danymi
if "data_dir" not in globals():
    data_dir = Path("data")

csv_path = data_dir / "transactions_with_explanations.csv"
report_path = data_dir / "aml_report.txt"

print(f"📄 Wczytuję raport z: {csv_path}")

# Ten string będziemy wypełniać treścią raportu, a na końcu zapiszemy do pliku TXT
report_text = ""

if not csv_path.exists():
    msg = "❌ Plik transactions_with_explanations.csv nie istnieje. Najpierw uruchom komórkę z batchem CSV."
    print(msg)
    report_text += msg + "\n"
else:
    with csv_path.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    if not rows:
        msg = "Plik jest pusty, brak transakcji do raportu."
        print(msg)
        report_text += msg + "\n"
    else:
        header = f"=== RAPORT AML – {len(rows)} transakcji ===\n\n"
        print(header)
        report_text += header

        for idx, row in enumerate(rows, start=1):
            # Parsowanie pól
            try:
                amount = int(row["amount"])
            except Exception:
                amount = row["amount"]

            try:
                tx_count_24h = int(row["tx_count_24h"])
            except Exception:
                tx_count_24h = row["tx_count_24h"]

            is_pep_raw = str(row.get("is_pep", "")).strip()
            is_pep_bool = is_pep_raw.lower() in {"true", "1", "yes", "y", "t"}

            decision = row.get("decision", "")
            status = row.get("decision_status", "")
            active_rules = row.get("active_rules") or ""
            explanation_text = row.get("explanation_pl", "").strip()

            # Formatowanie bloku raportu
            block = []
            block.append("=" * 70)
            block.append(f"📌 Transakcja {row.get('transaction_id', 'N/A')} | Klient: {row.get('customer_id', 'N/A')}")
            block.append(
                f"Kwota: {amount} | Liczba transakcji 24h: {tx_count_24h} | "
                f"PEP: {is_pep_bool} (raw: {is_pep_raw})"
            )
            block.append(f"Decyzja silnika: {decision} (status: {status})")

            if active_rules:
                block.append(f"Aktywne reguły: {active_rules}")
            else:
                block.append("Aktywne reguły: brak – żadna reguła nie została uruchomiona (case CLEAN).")

            block.append("\nUzasadnienie (pełne wyjaśnienie):")
            block.append(explanation_text or "(brak wyjaśnienia w pliku)")
            block.append("")  # pusta linia

            block_text = "\n".join(block)
            print(block_text)

            report_text += block_text + "\n"

        footer = "\n" + "=" * 70 + "\n🏁 Koniec raportu AML.\n"
        print(footer)
        report_text += footer

# Zapis do pliku TXT
with report_path.open("w", encoding="utf-8") as f:
    f.write(report_text)

print(f"\n💾 Zapisano raport AML do pliku: {report_path}")


📄 Wczytuję raport z: data/transactions_with_explanations.csv
=== RAPORT AML – 208 transakcji ===


📌 Transakcja T0001 | Klient: C0001
Kwota: 15000 | Liczba transakcji 24h: 1 | PEP: False (raw: False)
Decyzja silnika: FLAGGED (status: SAT)
Aktywne reguły: fraud.high_amount

Uzasadnienie (pełne wyjaśnienie):
Decyzja: transakcja została OFLAGOWANA (FLAGGED). Kluczowe dane wejściowe: amount=15000, is_pep=False, tx_count_24h=1.

Powody (aktywne reguły):
- Reguła 'fraud.high_amount': IF amount > 10000 THEN is_suspicious = TRUE

Reguły, które nie zadziałały w tym przypadku:
- Reguła 'fraud.velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_24h > 5 THEN is_suspicious = TRUE
- Reguła 'fraud.ultra_high_amount_extreme_velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF amount > 1000000 AND tx_count_24h > 100 THEN is_suspicious = TRUE
- Reguła 'fraud.pep_high_risk' była spełniona logicznie, ale jej warunek nie dotyczył tego 

In [9]:
# Cell: Rule Insights – definicje (FAZA 4.2)

from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Dict, List, Literal, Optional, Tuple

from rule_analytics import (
    DecisionLogReader,
    DecisionOutcomeStats,
    RuleAnalyticsResult,
    RuleStats,
)

LanguageCode = Literal["pl", "en"]


@dataclass
class RuleInsight:
    """Wniosek dotyczący pojedynczej reguły na podstawie statystyk z logów."""

    rule_id: str
    description: Optional[str]
    activity_rate: float
    coverage_rate: float
    conflict_rate: float
    flagged_activation_rate: float
    clean_activation_rate: float
    total_occurrences: int
    active: int
    inactive: int
    in_conflict: int
    classification: str
    insight_text: str


@dataclass
class SystemInsight:
    """Podsumowanie systemu regułowego jako całości."""

    total_decisions: int
    share_flagged: float
    share_clean: float
    share_other: float
    headline: str
    # Najważniejsze (najbardziej aktywne) reguły
    top_rules: List[str] = field(default_factory=list)
    # Reguły rzadko używane / martwe
    low_usage_rules: List[str] = field(default_factory=list)
    # Próg, wg którego liczymy „słabe użycie” (0.05 = 5%)
    low_usage_threshold: float = 0.0


@dataclass
class RuleInsightsReport:
    """Pełny raport wniosków z analizy reguł."""

    system: SystemInsight
    rules: Dict[str, RuleInsight] = field(default_factory=dict)

    def as_dict(self) -> Dict[str, Any]:
        return {
            "system": {
                "total_decisions": self.system.total_decisions,
                "share_flagged": self.system.share_flagged,
                "share_clean": self.system.share_clean,
                "share_other": self.system.share_other,
                "headline": self.system.headline,
                "top_rules": list(self.system.top_rules),
                "low_usage_rules": list(self.system.low_usage_rules),
                "low_usage_threshold": self.system.low_usage_threshold,
            },
            "rules": {
                rule_id: {
                    "rule_id": insight.rule_id,
                    "description": insight.description,
                    "activity_rate": insight.activity_rate,
                    "coverage_rate": insight.coverage_rate,
                    "conflict_rate": insight.conflict_rate,
                    "flagged_activation_rate": insight.flagged_activation_rate,
                    "clean_activation_rate": insight.clean_activation_rate,
                    "total_occurrences": insight.total_occurrences,
                    "active": insight.active,
                    "inactive": insight.inactive,
                    "in_conflict": insight.in_conflict,
                    "classification": insight.classification,
                    "insight_text": insight.insight_text,
                }
                for rule_id, insight in sorted(self.rules.items())
            },
        }


class RuleInsightsEngine:
    """Warstwa „Rule Insights” nad RuleAnalyticsResult.

    Parametry:
        language:
            "pl" lub "en" – język tekstowych insightów.
        high_activity_threshold:
            Próg aktywności, powyżej którego reguła jest klasyfikowana jako "core".
        medium_activity_threshold:
            Próg aktywności dla klasy "supporting".
        min_occurrences_for_insight:
            Minimalna liczba wystąpień, żeby w ogóle klasyfikować regułę.
        low_usage_activity_threshold:
            Próg aktywności, poniżej którego reguła jest traktowana jako
            „słabo używana” (np. 0.05 = 5% decyzji).
    """

    def __init__(
        self,
        *,
        language: LanguageCode = "pl",
        high_activity_threshold: float = 0.3,
        medium_activity_threshold: float = 0.1,
        min_occurrences_for_insight: int = 1,
        low_usage_activity_threshold: float = 0.05,
    ) -> None:
        self._language: LanguageCode = language
        self._high_activity_threshold: float = high_activity_threshold
        self._medium_activity_threshold: float = medium_activity_threshold
        self._min_occurrences_for_insight: int = min_occurrences_for_insight
        # Próg słabego użycia – bezpiecznie obcięty do [0.0, 1.0]
        self._low_usage_activity_threshold: float = max(
            0.0, min(low_usage_activity_threshold, 1.0)
        )

    def build_report(
        self,
        analytics: RuleAnalyticsResult,
        *,
        log_path: Optional[str] = None,
    ) -> RuleInsightsReport:
        """Buduje raport wniosków na podstawie RuleAnalyticsResult."""
        outcome_stats = analytics.outcome_stats
        total_decisions = max(outcome_stats.total_decisions, 1)

        flagged_total = outcome_stats.by_decision.get("FLAGGED", 0)
        clean_total = outcome_stats.by_decision.get("CLEAN", 0)
        other_total = total_decisions - flagged_total - clean_total

        if log_path is not None:
            (
                active_in_flagged,
                active_in_clean,
            ) = self._compute_activation_by_decision_outcome(
                log_path=log_path,
            )
        else:
            active_in_flagged, active_in_clean = {}, {}

        system_insight = self._build_system_insight(
            outcome_stats=outcome_stats,
            total_decisions=total_decisions,
            flagged_total=flagged_total,
            clean_total=clean_total,
            other_total=other_total,
            analytics=analytics,
        )

        rule_insights: Dict[str, RuleInsight] = {}

        for rule_id, stats in analytics.rule_stats.items():
            insight = self._build_rule_insight(
                rule_id=rule_id,
                stats=stats,
                outcome_stats=outcome_stats,
                total_decisions=total_decisions,
                flagged_total=flagged_total,
                clean_total=clean_total,
                active_in_flagged=active_in_flagged.get(rule_id, 0),
                active_in_clean=active_in_clean.get(rule_id, 0),
            )
            rule_insights[rule_id] = insight

        return RuleInsightsReport(system=system_insight, rules=rule_insights)

    def _build_system_insight(
        self,
        *,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        other_total: int,
        analytics: RuleAnalyticsResult,
    ) -> SystemInsight:
        share_flagged = flagged_total / total_decisions
        share_clean = clean_total / total_decisions
        share_other = max(other_total, 0) / total_decisions

        if self._language == "pl":
            headline = (
                f"System przetworzył {total_decisions} decyzji. "
                f"{share_clean:.1%} transakcji zostało ocenionych jako CZYSTE, "
                f"{share_flagged:.1%} jako OFLAGOWANE, "
                f"{share_other:.1%} to pozostałe statusy."
            )
        else:
            headline = (
                f"The system processed {total_decisions} decisions. "
                f"{share_clean:.1%} of transactions were marked as CLEAN, "
                f"{share_flagged:.1%} as FLAGGED, "
                f"{share_other:.1%} had other statuses."
            )

        # TOP reguły po liczbie aktywacji
        top_rules_stats = sorted(
            analytics.rule_stats.values(),
            key=lambda r: r.active,
            reverse=True,
        )[:5]

        top_rule_lines: List[str] = []
        for stats in top_rules_stats:
            if total_decisions == 0:
                activity_pct = 0.0
            else:
                activity_pct = stats.active / total_decisions
            if self._language == "pl":
                line = (
                    f"- Reguła '{stats.rule_id}' była aktywna "
                    f"w {activity_pct:.1%} wszystkich decyzji "
                    f"({stats.active}/{total_decisions})."
                )
            else:
                line = (
                    f"- Rule '{stats.rule_id}' was active in "
                    f"{activity_pct:.1%} of all decisions "
                    f"({stats.active}/{total_decisions})."
                )
            top_rule_lines.append(line)

        # Reguły rzadko używane / martwe
        low_usage_lines: List[str] = []

        coverage = analytics.coverage_report
        # 1) Reguły enabled=True, które w ogóle nie pojawiły się w logach
        if coverage is not None and coverage.unused_rules:
            for rule_id in coverage.unused_rules:
                if self._language == "pl":
                    line = (
                        f"- Reguła '{rule_id}' nie pojawiła się w żadnej decyzji "
                        "(0.0% wszystkich decyzji) – kandydatka na martwą lub "
                        "wymagającą dopasowania."
                    )
                else:
                    line = (
                        f"- Rule '{rule_id}' did not appear in any decision "
                        "(0.0% of all decisions) – candidate for deprecation or "
                        "redesign."
                    )
                low_usage_lines.append(line)

        # 2) Reguły, które są w statystykach, ale mają niski poziom aktywności
        if total_decisions > 0:
            for stats in sorted(
                analytics.rule_stats.values(),
                key=lambda r: r.active,
            ):
                activity_pct = stats.active / total_decisions

                # Jeżeli mamy też unused_rules z coverage, unikamy dublowania
                if activity_pct == 0.0:
                    if coverage is not None and stats.rule_id in coverage.unused_rules:
                        continue

                if activity_pct <= self._low_usage_activity_threshold:
                    if self._language == "pl":
                        line = (
                            f"- Reguła '{stats.rule_id}' była aktywna tylko w "
                            f"{activity_pct:.1%} decyzji "
                            f"({stats.active}/{total_decisions})."
                        )
                    else:
                        line = (
                            f"- Rule '{stats.rule_id}' was active in only "
                            f"{activity_pct:.1%} of decisions "
                            f"({stats.active}/{total_decisions})."
                        )
                    low_usage_lines.append(line)

        return SystemInsight(
            total_decisions=total_decisions,
            share_flagged=share_flagged,
            share_clean=share_clean,
            share_other=share_other,
            headline=headline,
            top_rules=top_rule_lines,
            low_usage_rules=low_usage_lines,
            low_usage_threshold=self._low_usage_activity_threshold,
        )

    def _build_rule_insight(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
    ) -> RuleInsight:
        occurrences = stats.total_occurrences
        active = stats.active
        inactive = stats.inactive
        in_conflict = stats.in_conflict

        activity_rate = active / total_decisions if total_decisions else 0.0
        coverage_rate = (
            occurrences / total_decisions if total_decisions else 0.0
        )
        conflict_rate = (
            in_conflict / total_decisions if total_decisions else 0.0
        )

        flagged_activation_rate = (
            active_in_flagged / flagged_total if flagged_total else 0.0
        )
        clean_activation_rate = (
            active_in_clean / clean_total if clean_total else 0.0
        )

        classification = self._classify_rule(
            occurrences=occurrences,
            activity_rate=activity_rate,
            conflict_rate=conflict_rate,
        )

        insight_text = self._render_rule_insight_text(
            rule_id=rule_id,
            stats=stats,
            total_decisions=total_decisions,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_total=flagged_total,
            clean_total=clean_total,
            active_in_flagged=active_in_flagged,
            active_in_clean=active_in_clean,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            classification=classification,
        )

        return RuleInsight(
            rule_id=rule_id,
            description=stats.description,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            total_occurrences=occurrences,
            active=active,
            inactive=inactive,
            in_conflict=in_conflict,
            classification=classification,
            insight_text=insight_text,
        )

    def _classify_rule(
        self,
        *,
        occurrences: int,
        activity_rate: float,
        conflict_rate: float,
    ) -> str:
        if occurrences < self._min_occurrences_for_insight:
            return "unused"
        if conflict_rate > 0.05:
            return "conflict_prone"
        if activity_rate >= self._high_activity_threshold:
            return "core"
        if activity_rate >= self._medium_activity_threshold:
            return "supporting"
        return "niche"

    def _render_rule_insight_text(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        total_decisions: int,
        activity_rate: float,
        coverage_rate: float,
        conflict_rate: float,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
        flagged_activation_rate: float,
        clean_activation_rate: float,
        classification: str,
    ) -> str:
        total_decisions = max(total_decisions, 1)

        if self._language == "pl":
            parts: List[str] = []

            parts.append(
                f"Reguła '{rule_id}' pojawiła się w "
                f"{coverage_rate:.1%} wszystkich decyzji "
                f"({stats.total_occurrences}/{total_decisions})."
            )
            parts.append(
                f"Aktywnie zadziałała w {activity_rate:.1%} decyzji "
                f"({stats.active}/{total_decisions})."
            )

            if flagged_total > 0 and active_in_flagged > 0:
                parts.append(
                    f"Była aktywna w {flagged_activation_rate:.1%} wszystkich decyzji "
                    f"OFLAGOWANYCH ({active_in_flagged}/{flagged_total})."
                )

            if clean_total > 0 and active_in_clean > 0:
                parts.append(
                    f"Była aktywna w {clean_activation_rate:.1%} wszystkich decyzji "
                    f"CLEAN ({active_in_clean}/{clean_total})."
                )

            if stats.in_conflict > 0:
                parts.append(
                    f"W {stats.in_conflict} decyzjach reguła znalazła się w jądrze UNSAT, "
                    f"co stanowi {conflict_rate:.1%} wszystkich decyzji – kandydat do przeglądu."
                )

            classification_comment = self._classification_comment_pl(
                classification=classification,
            )
            if classification_comment:
                parts.append(classification_comment)

            return " ".join(parts)

        parts_en: List[str] = []

        parts_en.append(
            f"Rule '{rule_id}' appeared in "
            f"{coverage_rate:.1%} of all decisions "
            f"({stats.total_occurrences}/{total_decisions})."
        )
        parts_en.append(
            f"It was actively triggered in {activity_rate:.1%} of decisions "
            f"({stats.active}/{total_decisions})."
        )

        if flagged_total > 0 and active_in_flagged > 0:
            parts_en.append(
                f"It was active in {flagged_activation_rate:.1%} of all FLAGGED decisions "
                f"({active_in_flagged}/{flagged_total})."
            )

        if clean_total > 0 and active_in_clean > 0:
            parts_en.append(
                f"It was active in {clean_activation_rate:.1%} of all CLEAN decisions "
                f"({active_in_clean}/{clean_total})."
            )

        if stats.in_conflict > 0:
            parts_en.append(
                f"In {stats.in_conflict} decisions the rule contributed to UNSAT core "
                f"({conflict_rate:.1%} of all decisions) – candidate for review."
            )

        classification_comment_en = self._classification_comment_en(
            classification=classification,
        )
        if classification_comment_en:
            parts_en.append(classification_comment_en)

        return " ".join(parts_en)

    def _classification_comment_pl(self, classification: str) -> str:
        if classification == "core":
            return (
                "Jest to jedna z kluczowych reguł systemu – wysoka aktywność i "
                "istotny wpływ na decyzje."
            )
        if classification == "supporting":
            return (
                "Reguła wspierająca: używana regularnie, ale nie dominuje w systemie."
            )
        if classification == "niche":
            return (
                "Reguła niszowa: działa rzadko, może opisywać specjalne scenariusze "
                "lub wymaga przeglądu progów."
            )
        if classification == "conflict_prone":
            return (
                "Reguła często uczestniczy w konfliktach (UNSAT) – silna rekomendacja "
                "do przeglądu logicznego."
            )
        if classification == "unused":
            return (
                "Reguła praktycznie nie występuje w logach – kandydatka na martwą lub "
                "wymagającą dopasowania."
            )
        return ""

    def _classification_comment_en(self, classification: str) -> str:
        if classification == "core":
            return (
                "This is one of the core rules – high activity and significant impact "
                "on decisions."
            )
        if classification == "supporting":
            return (
                "Supporting rule: used regularly but does not dominate the system."
            )
        if classification == "niche":
            return (
                "Niche rule: rarely triggers; may describe special scenarios or "
                "require threshold tuning."
            )
        if classification == "conflict_prone":
            return (
                "The rule frequently appears in UNSAT cores – strong candidate for "
                "logical review."
            )
        if classification == "unused":
            return (
                "The rule is practically unused in logs – candidate for deprecation or "
                "redesign."
            )
        return ""

    def _compute_activation_by_decision_outcome(
        self,
        *,
        log_path: str,
    ) -> Tuple[Dict[str, int], Dict[str, int]]:
        active_in_flagged: Dict[str, int] = {}
        active_in_clean: Dict[str, int] = {}

        reader = DecisionLogReader(log_path)

        for record in reader.iter_decisions():
            bundle = record.bundle
            decision = str(bundle.get("decision", "UNKNOWN"))
            active_axioms = bundle.get("active_axioms") or []

            active_ids_in_decision = {
                str(entry.get("id", ""))
                for entry in active_axioms
                if entry.get("id")
            }

            if decision == "FLAGGED":
                target = active_in_flagged
            elif decision == "CLEAN":
                target = active_in_clean
            else:
                target = None

            if target is None:
                continue

            for rule_id in active_ids_in_decision:
                target[rule_id] = target.get(rule_id, 0) + 1

        return active_in_flagged, active_in_clean


In [10]:
# Cell: Rule Insights – wnioski z FAZY 4.2 na bazie analytics_result

# Próg słabego użycia (np. 0.05 = 5% decyzji)
insights_engine = RuleInsightsEngine(
    language="pl",
    low_usage_activity_threshold=0.05,
)

insights_report = insights_engine.build_report(
    analytics_result,
    log_path=str(logs_dir / "fraud_rules_demo.jsonl"),
)

print("=== INSIGHT: PODSUMOWANIE SYSTEMU ===")
print(insights_report.system.headline)
print()

print("Najważniejsze reguły:")
for line in insights_report.system.top_rules:
    print(line)

if insights_report.system.low_usage_rules:
    threshold_pct = insights_report.system.low_usage_threshold * 100.0
    print(
        f"\nReguły rzadko używane (aktywność ≤ {threshold_pct:.1f}% decyzji):"
    )
    for line in insights_report.system.low_usage_rules:
        print(line)

print("\n=== INSIGHT: WNIOSKI PER REGUŁA ===")
for rule_id, insight in sorted(insights_report.rules.items()):
    print(f"\n--- {rule_id} ---")
    print(insight.insight_text)


=== INSIGHT: PODSUMOWANIE SYSTEMU ===
System przetworzył 1449 decyzji. 22.6% transakcji zostało ocenionych jako CZYSTE, 77.4% jako OFLAGOWANE, 0.0% to pozostałe statusy.

Najważniejsze reguły:
- Reguła 'fraud.high_amount' była aktywna w 45.2% wszystkich decyzji (655/1449).
- Reguła 'fraud.velocity' była aktywna w 32.9% wszystkich decyzji (476/1449).
- Reguła 'fraud.pep_high_risk' była aktywna w 21.5% wszystkich decyzji (311/1449).
- Reguła 'fraud.ultra_high_amount_extreme_velocity' była aktywna w 0.0% wszystkich decyzji (0/1449).

Reguły rzadko używane (aktywność ≤ 5.0% decyzji):
- Reguła 'fraud.ultra_high_amount_extreme_velocity' była aktywna tylko w 0.0% decyzji (0/1449).

=== INSIGHT: WNIOSKI PER REGUŁA ===

--- fraud.high_amount ---
Reguła 'fraud.high_amount' pojawiła się w 100.0% wszystkich decyzji (1449/1449). Aktywnie zadziałała w 45.2% decyzji (655/1449). Była aktywna w 58.4% wszystkich decyzji OFLAGOWANYCH (655/1121). Jest to jedna z kluczowych reguł systemu – wysoka aktywność

In [11]:
# Cell: Rule Gaps – analiza segmentów transakcji (FAZA 4.3)

from rule_gaps import RuleGapsConfig, RuleGapsEngine

csv_path = data_dir / "transactions_with_explanations.csv"

gaps_engine = RuleGapsEngine(
    RuleGapsConfig(
        min_total_cases=20,       # minimalna liczba case'ów w segmencie
        min_flagged_rate=0.3,     # min. 30% FLAGGED, żeby segment był "gorący"
        max_distinct_rules_in_gap=2,    # max 1-2 reguły w oflagowanych
        min_dominant_rule_share=0.6,    # dominująca reguła >= 60% FLAGGED
    )
)

gaps_result = gaps_engine.analyze_csv(csv_path)

print(f"📊 Rule Gaps – analiza segmentów na podstawie: {csv_path}")
print(f"  Łączna liczba segmentów: {len(gaps_result.segments)}")
print(f"  Segmenty zidentyfikowane jako potencjalne luki: {len(gaps_result.gap_segments)}\n")

for idx, gap in enumerate(gaps_result.gap_segments, start=1):
    print("=" * 70)
    print(f"🧩 GAP #{idx}: {gap.key.label()}")
    print(
        f"Liczba decyzji w segmencie: {gap.total} "
        f"(FLAGGED: {gap.flagged}, CLEAN: {gap.clean}, "
        f"FLAGGED %: {gap.flagged_rate:.1%})"
    )

    if gap.dominant_rules:
        rules_list = ", ".join(gap.dominant_rules)
        print(f"Dominujące reguły w oflagowanych decyzjach: {rules_list}")
        print(
            "Udział najsilniejszej reguły we FLAGGED: "
            f"{gap.dominant_rules_share:.1%}"
        )
    else:
        print("W oflagowanych decyzjach nie aktywowała się żadna reguła.")

    print("\nWniosek:")
    print(gap.note_text)
    print()

if not gaps_result.gap_segments:
    print("✅ Brak oczywistych luk – żaden segment nie spełnia kryteriów GAP.")


📊 Rule Gaps – analiza segmentów na podstawie: data/transactions_with_explanations.csv
  Łączna liczba segmentów: 27
  Segmenty zidentyfikowane jako potencjalne luki: 3

🧩 GAP #1: amount=[20k, 100k), tx_count_24h=2–5, non-PEP
Liczba decyzji w segmencie: 28 (FLAGGED: 28, CLEAN: 0, FLAGGED %: 100.0%)
Dominujące reguły w oflagowanych decyzjach: fraud.high_amount
Udział najsilniejszej reguły we FLAGGED: 100.0%

Wniosek:
Segment amount=[20k, 100k), tx_count_24h=2–5, non-PEP ma 28 decyzji, z czego 28 (100.0%) jest OFLAGOWANYCH. W oflagowanych decyzjach aktywuje się bardzo ograniczony zestaw reguł (łącznie 1), z dominującymi: fraud.high_amount (28 razy). Najmocniejsza reguła pokrywa około 100.0% wszystkich flagowań w tym segmencie. To sugeruje, że warto rozważyć doprecyzowanie logiki dla tego segmentu (np. rozbicie na bardziej szczegółowe reguły) lub sprawdzenie, czy brak dodatkowych reguł nie powoduje nadmiernego obciążenia pojedynczej reguły.

🧩 GAP #2: amount=[1k, 5k), tx_count_24h=6–10, no

In [14]:
# Cell: Candidate Rule Engine – generowanie propozycji reguł (FAZA 5)

from src.candidate_rule_engine import (
    CandidateRuleConfig,
    CandidateRuleEngine,
)

# CSV z decyzjami + wyjaśnieniami (wynik batch scoringu)
csv_candidates_path = data_dir / "transactions_with_explanations.csv"

candidate_config = CandidateRuleConfig(
    min_triggered_cases=10,  # możesz podnieść np. do 20 w realnym banku
)

candidate_engine = CandidateRuleEngine(
    schema=schema,
    decision_field="is_suspicious",
    ruleset_path=fraud_rules_path,
    config=candidate_config,
)

candidates = candidate_engine.generate_candidates_from_gaps(
    gaps_result,
    csv_path=csv_candidates_path,
)

print(
    f"📌 Wygenerowano {len(candidates)} kandydatów reguł "
    f"na podstawie {len(gaps_result.gap_segments)} segmentów GAP."
)

for idx, candidate in enumerate(candidates, start=1):
    metrics = candidate.metrics
    proof = candidate.proof

    print("\n" + "=" * 70)
    print(f"🧪 Kandydat #{idx}: {candidate.rule_id}")
    print(f"Segment: {candidate.segment.key.label()}")
    print(f"Reguła (NL): {candidate.nl_rule_text}")
    print(f"Opis: {candidate.description}\n")

    print(
        f"- Transakcje w segmencie (z Rule Gaps): "
        f"{metrics.segment_total} (FLAGGED: {metrics.segment_flagged}, "
        f"CLEAN: {metrics.segment_clean}, "
        f"FLAGGED%: {metrics.segment_flagged_rate:.1%})"
    )
    print(
        f"- Transakcje spełniające warunek kandydata w całym zbiorze: "
        f"{metrics.triggered_total} / {metrics.total_cases} "
        f"({metrics.triggered_share:.1%})"
    )
    print(
        f"- W obszarze działania reguły: "
        f"FLAGGED={metrics.triggered_flagged}, "
        f"CLEAN={metrics.triggered_clean}, "
        f"inne={metrics.triggered_other}"
    )

    if proof.is_conflict_free:
        print("- Spójność z istniejącym rulesetem: ✅ brak konfliktów w Z3")
    else:
        print(
            f"- Spójność z istniejącym rulesetem: ⚠️ {proof.conflict_count} "
            "potencjalnych konfliktów z istniejącymi regułami:"
        )
        for detail in proof.conflict_details:
            print(f"  • {detail}")


📌 Wygenerowano 3 kandydatów reguł na podstawie 3 segmentów GAP.

🧪 Kandydat #1: fraud.candidate_gap.001
Segment: amount=[20k, 100k), tx_count_24h=2–5, non-PEP
Reguła (NL): IF amount >= 20000 AND amount < 100000 AND tx_count_24h >= 2 AND tx_count_24h <= 5 AND is_pep == FALSE THEN is_suspicious = TRUE
Opis: Kandydat reguły dla segmentu: amount=[20k, 100k), tx_count_24h=2–5, non-PEP (automatycznie wygenerowany na podstawie Rule Gaps).

- Transakcje w segmencie (z Rule Gaps): 28 (FLAGGED: 28, CLEAN: 0, FLAGGED%: 100.0%)
- Transakcje spełniające warunek kandydata w całym zbiorze: 28 / 208 (13.5%)
- W obszarze działania reguły: FLAGGED=28, CLEAN=0, inne=0
- Spójność z istniejącym rulesetem: ✅ brak konfliktów w Z3

🧪 Kandydat #2: fraud.candidate_gap.002
Segment: amount=[1k, 5k), tx_count_24h=6–10, non-PEP
Reguła (NL): IF amount >= 1000 AND amount < 5000 AND tx_count_24h >= 6 AND tx_count_24h <= 10 AND is_pep == FALSE THEN is_suspicious = TRUE
Opis: Kandydat reguły dla segmentu: amount=[1k, 5k